# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [12]:
# Your solution goes here
!pip install requests beautifulsoup4

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import time

In [14]:


# Mapping textual ratings to numeric values
RATING_MAP = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}

BASE_URL = "http://books.toscrape.com/"

def get_genre_and_description(book_url):
    """Fetch genre and description from the book detail page."""
    response = requests.get(book_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Genre is in the breadcrumb, third item
    genre = soup.select('ul.breadcrumb li a')[-1].text.strip()

    # Description is under the 'product_page' section
    description_tag = soup.find('meta', attrs={'name': 'description'})
    description = description_tag['content'].strip() if description_tag else 'No description available.'

    # UPC is found in the product table
    upc = soup.find('th', string='UPC').find_next_sibling('td').text.strip()

    return genre, description, upc

def scrape_books(min_rating=4, max_price=20):
    """Scrape books with rating >= min_rating and price <= max_price."""
    books_data = []
    page_num = 1

    while True:
        url = f"{BASE_URL}catalogue/page-{page_num}.html" if page_num > 1 else f"{BASE_URL}catalogue/page-1.html"
        response = requests.get(url)

        if response.status_code != 200:
            break  # Stop if the page doesn't exist

        soup = BeautifulSoup(response.content, 'html.parser')
        books = soup.find_all('article', class_='product_pod')

        for book in books:
            title = book.h3.a['title']
            price_text = book.find('p', class_='price_color').text
            price = float(price_text.replace('£', '').strip())
            availability = book.find('p', class_='instock availability').text.strip()

            # Get rating as number
            rating_class = book.find('p', class_='star-rating')['class'][1]
            rating = RATING_MAP.get(rating_class, 0)

            # Filter by rating and price
            if rating >= min_rating and price <= max_price:
                book_url = BASE_URL + 'catalogue/' + book.h3.a['href'].replace('../../../', '')
                genre, description, upc = get_genre_and_description(book_url)

                books_data.append({
                    'UPC': upc,
                    'Title': title,
                    'Price (£)': price,
                    'Rating': rating,
                    'Genre': genre,
                    'Availability': availability,
                    'Description': description
                })

        page_num += 1

    return pd.DataFrame(books_data)


In [15]:
df = scrape_books(min_rating=4, max_price=20)
print(df.head())


                UPC                                              Title  \
0  ce6396b0f23f6ecc                                        Set Me Free   
1  6258a1f6a6dcfe50  The Four Agreements: A Practical Guide to Pers...   
2  6be3beb0793a53e7                                     Sophie's World   
3  657fe5ead67a7767            Untitled Collection: Sabbath Poems 2014   
4  51653ef291ab7ddc                                    This One Summer   

   Price (£)  Rating           Genre Availability  \
0      17.46       5     Young Adult     In stock   
1      17.66       5    Spirituality     In stock   
2      15.94       5      Philosophy     In stock   
3      14.27       4          Poetry     In stock   
4      19.49       4  Sequential Art     In stock   

                                         Description  
0  Aaron Ledbetter’s future had been planned out ...  
1  In The Four Agreements, don Miguel Ruiz reveal...  
2  A page-turning novel that is also an explorati...  
3  More than thi

In [16]:
from IPython.display import display

# Display the DataFrame as a table
display(df)

,UPC,Title,Price (£),Rating,Genre,Availability,Description
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,In stock,Aaron Ledbetter’s future had been planned out ...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,In stock,"In The Four Agreements, don Miguel Ruiz reveal..."
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,In stock,A page-turning novel that is also an explorati...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,In stock,"More than thirty-five years ago, when the weat..."
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,In stock,"Every summer, Rose goes with her mom and dad t..."
...,...,...,...,...,...,...,...
70,9c96cd1329fbd82d,The Zombie Room,19.69,5,Default,In stock,An unlikely bond is forged between three men f...
71,b78deb463531d078,The Silent Wife,12.34,5,Fiction,In stock,A chilling psychological thriller about a marr...
72,4280ac3eab57aa5d,The Girl You Lost,12.29,5,Mystery,In stock,Eighteen years ago your baby daughter was snat...
73,29fc016c459aeb14,The Edge of Reason (Bridget Jones #2),19.18,4,Womens Fiction,In stock,Monday 27 January“7:15 a.m. Hurrah! The wilder...
